# Sincere_etl_data_wrangling 

In [1]:
# Importing needed libraries
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from pandas.io.json import json_normalize
import json


# Blocking warnings
import warnings
warnings.filterwarnings('ignore')

## A) Using OMDb API to extract IMDb data

Considering 62.000 movies of movilens dataset, the IMDB aditional movie's information extraction demanded time, computational resources and patience. If you want to skip this part of notebook, start from section B. 

#### MovieLens links.csv

Identifiers that can be used to link to other sources of movie data are contained in the file links.csv. Each line of this file after the header row represents one movie, and has the following format:
movieId,imdbId,tmdbId
movieId is an identifier for movies used by https://movielens.org. E.g., the movie Toy Story has the link https://movielens.org/movies/1.
imdbId is an identifier for movies used by http://www.imdb.com. E.g., the movie Toy Story has the link http://www.imdb.com/title/tt0114709/.
tmdbId is an identifier for movies used by https://www.themoviedb.org. E.g., the movie Toy Story has the link https://www.themoviedb.org/movie/862.


#### Movie's aditional information extraction
Step by step data collection using the OMDb API - http://www.omdbapi.com/ 
More information can be found in the Databases - Readme.txt.

* We've decided to use the IMDb database to provide more information about the movies listed on the original MovieLens dataset. Using OMDb API, we've been able to extract the following information:
    * Title: Movie/Tv Show title.
    * Year: Year of production. 
    * Released: Year of release.
    * Genre: List of move genres.
    * Director: Name of director.
    * Writer: List of Writers.
    * Actors: Name of the leading star.
    * Language: Language of origin.
    * Country: Country of origin (production).
    * Awards:Awards nominated and received.
    * imdbRating: movie rate based on IMDB users. 
    * imdbVotes: number of votes a movie received. 
    * imdbID id of a movie on IDMB database.

Our intention is to provide more information to our models in hopes of achieving better clustering results.

In [2]:
'''
# importing Movilens data with movie codes to be used as key to IMDB database
df_links = pd.read_csv('links.csv')

# function to receive api key and change movie id inside the loop
def getUrl(search_id):
    url = 'http://www.omdbapi.com/?apikey={}&i=tt{}'.format(api_key, search_id)
    return url

#API keys
api_key = #Insert API key

#For imdb scrapping purposes

df_links_column = list(df_links['imdbId'])
df_links_column = [str(item).zfill(8) for item in df_links_column]

print(len(df_links_column))
df_links_column

#loop to run selected movie ids and append the returning movie information dict in a list
imdb_info = []
for i in range(len(df_links_column)):
    results = requests.get(getUrl(df_links_column[i])).json()
    imdb_info.append(results)

#Checking NaN Titles
sum(df_imdb.Title.isnull())

#Transforming into Dataframe from Dict
df_imdb = pd.DataFrame.from_dict(imdb_info)

#Saving into a csv
df_imdb.to_csv('df_imdb.csv', index = False)
'''

"\n# importing Movilens data with movie codes to be used as key to IMDB database\ndf_links = pd.read_csv('links.csv')\n\n# function to receive api key and change movie id inside the loop\ndef getUrl(search_id):\n    url = 'http://www.omdbapi.com/?apikey={}&i=tt{}'.format(api_key, search_id)\n    return url\n\n#API keys\napi_key = #Insert API key\n\n#For imdb scrapping purposes\n\ndf_links_column = list(df_links['imdbId'])\ndf_links_column = [str(item).zfill(8) for item in df_links_column]\n\nprint(len(df_links_column))\ndf_links_column\n\n#loop to run selected movie ids and append the returning movie information dict in a list\nimdb_info = []\nfor i in range(len(df_links_column)):\n    results = requests.get(getUrl(df_links_column[i])).json()\n    imdb_info.append(results)\n\n#Checking NaN Titles\nsum(df_imdb.Title.isnull())\n\n#Transforming into Dataframe from Dict\ndf_imdb = pd.DataFrame.from_dict(imdb_info)\n\n#Saving into a csv\ndf_imdb.to_csv('df_imdb.csv', index = False)\n"

## B) IMDB Data Wrangling

In [3]:
# shortcut | reading from stored intermediate results
df_imdb = pd.read_csv('df_imdb.csv')

In [4]:
# creating Dataframe with only target columns
df_imdb = df_imdb[['imdbID','Genre','Awards','Year','imdbRating','imdbVotes']]

# adjusting imdbID to match movielens dataset
df_imdb['imdbID'] = df_imdb['imdbID'].str.replace('tt','').astype('int64')
df_imdb.rename(columns = {'imdbID': 'imdbId'}, inplace = True)

In [5]:
# importing movilens data to insert movieId on IMDB dataset
df_links = pd.read_csv('links.csv')
df_links = df_links[['movieId', 'imdbId']]
df_imdb = df_imdb.merge(df_links, on='imdbId', how='left')

# checking results
df_imdb.head(2)

,imdbId,Genre,Awards,Year,imdbRating,imdbVotes,movieId
0,114709,"Animation, Adventure, Comedy, Family, Fantasy",Nominated for 3 Oscars. Another 27 wins & 20 n...,1995,8.3,"904,049",1.0
1,113497,"Adventure, Comedy, Family, Fantasy",4 wins & 11 nominations.,1995,7.0,"314,983",2.0


### Wrangling Awards

In [6]:
# selecting necessary column
df_imdb_awards = df_imdb[['Awards']]

# separating words from the string and adding values to each category (oscar, win, nominated)
df_imdb_awards['oscar'] = np.where(df_imdb['Awards'].str.contains("osca", case=False, na=False), 5, 0)
df_imdb_awards['win'] = np.where(df_imdb['Awards'].str.contains("win", case=False, na=False), 3, 0)
df_imdb_awards['nominated'] = np.where(df_imdb['Awards'].str.contains("nominat", case=False, na=False), 1, 0)

# loop to ensure nominated fields have proper values
for i in range(len(df_imdb_awards['nominated'])):
    if df_imdb_awards['nominated'][i] == 0:
        if df_imdb_awards['win'][i] != 0:
            df_imdb_awards['nominated'][i] = 1
        elif df_imdb_awards['oscar'][i] != 0:
            df_imdb_awards['nominated'][i] = 1

# creating a new column for added values
df_imdb_awards['awards_total'] = df_imdb_awards['oscar'] + df_imdb_awards['win'] + df_imdb_awards['nominated']

# dropping unecessary columns
df_imdb_awards.drop(['Awards','nominated', 'oscar', 'win'], axis = 1, inplace = True)

# adding awards count column to main dataframe
df_imdb_expanded = df_imdb.join(df_imdb_awards) 

# dropping original awards column from main dataframe
df_imdb_expanded.drop('Awards', axis = 1, inplace = True)

### Genre wrangling

In [7]:
# selecting necessary columns
df_imdb_genre = df_imdb[['imdbId','Genre']]

# generating dummies for each genre
df_imdb_genre = df_imdb_genre.join(df_imdb_genre['Genre'].str.get_dummies(sep=', '))
df_imdb_genre.drop('Genre', axis = 1, inplace = True)
df_imdb_genre.drop('imdbId', axis = 1, inplace = True)

# dropping unecessary column and joining with the main imdb dataset
df_imdb_expanded.drop('Genre', axis = 1, inplace = True)
df_imdb_expanded = df_imdb_expanded.join(df_imdb_genre)

# converting year information to int
df_imdb_expanded['Year'] = df_imdb_expanded['Year'].str[:4]
df_imdb_expanded['Year'] = df_imdb_expanded['Year'].astype(int)

# storing intermediate results
df_imdb_expanded.reset_index(drop=True)
df_imdb_expanded.to_csv('df_imdb_expanded.csv', index = False)

# checking results
df_imdb_expanded.head(2)

,imdbId,Year,imdbRating,imdbVotes,movieId,awards_total,Action,Adult,Adventure,Animation,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,114709,1995,8.3,"904,049",1.0,9,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,113497,1995,7.0,"314,983",2.0,4,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# generating a dataset to use on movies wrangling due to Movielens database year missing data
df_imdb_year = df_imdb_expanded[['movieId','Year']]

# creating a column to calculate the age of the movie in 2019
df_imdb_year.rename(columns = {'Year':'year_imdb'}, inplace=True)
df_imdb_year['age_movie_in_2019'] = 2019 - df_imdb_year['year_imdb']

# storing result in a csv file
df_imdb_year.to_csv('df_imdb_year.csv', index = False)

# checking results
df_imdb_year.head(2)

,movieId,year_imdb,age_movie_in_2019
0,1.0,1995,24
1,2.0,1995,24


## C) movies.csv wrangling

In [9]:
# importing data
df_movies = pd.read_csv('movies.csv')

In [10]:
# creating a column to store the year of the movie and checking if there are null values
df_movies['year'] = df_movies.title.str.extract("\((\d{4})\)", expand=True)

# taking out the year from the title
df_movies['title'] = df_movies.title.str.replace('(\(\d\d\d\d\))', '')
df_movies['title'] = df_movies['title'].apply(lambda x: x.strip())

# dropping unecessary columns (they will be brought back from IMDB database later)
df_movies.drop(['genres','year'], inplace=True, axis=1)

# checking results
df_movies.head(2)

,movieId,title
0,1,Toy Story
1,2,Jumanji


## D) Adjusting ratings timestamp information

In [11]:
# importing data
df_ratings = pd.read_csv('ratings.csv')

# creating a new dataframe based on df ratings
df_ratings_expanded = df_ratings

In [12]:
# adjusting date information format
# creating colums to store rating hour and weekday
df_ratings_expanded["hour"] = pd.to_datetime(df_ratings_expanded['timestamp'],origin=pd.Timestamp('1970-01-01'), unit='s').dt.strftime('%H').astype(int)
df_ratings_expanded["weekday"] = pd.to_datetime(df_ratings_expanded['timestamp'],origin=pd.Timestamp('1970-01-01'), unit='s').dt.strftime('%w')
df_ratings_expanded["year_rated"] = pd.to_datetime(df_ratings_expanded['timestamp'],origin=pd.Timestamp('1970-01-01'), unit='s').dt.strftime('%Y')

# dropping unecessary column
df_ratings_expanded.drop('timestamp', inplace=True, axis=1)

# adding label for period of the day according with the rating hours
conditionlist = [
    (df_ratings_expanded['hour'] >= 6) & (df_ratings_expanded['hour'] < 12), # morning = 1
    (df_ratings_expanded['hour'] >= 12) & (df_ratings_expanded['hour'] < 18), # afternoon = 2
    (df_ratings_expanded['hour'] >= 18) & (df_ratings_expanded['hour'] <= 23), # evening = 3
    (df_ratings_expanded['hour'] >= 0) & (df_ratings_expanded['hour'] < 6)] # night = 4
choicelist = [1, 2, 3, 4]
df_ratings_expanded['time_of_day'] = np.select(conditionlist, choicelist, default='Not Specified')

# checking results
df_ratings_expanded

,userId,movieId,rating,hour,weekday,year_rated,time_of_day
0,1,296,5.0,15,3,2006,2
1,1,306,3.5,12,3,2006,2
2,1,307,5.0,12,3,2006,2
3,1,665,5.0,15,3,2006,2
4,1,899,3.5,12,3,2006,2
...,...,...,...,...,...,...,...
25000090,162541,50872,4.5,21,2,2009,3
25000091,162541,55768,2.5,20,2,2009,3
25000092,162541,56176,2.0,20,2,2009,3
25000093,162541,58559,4.0,21,2,2009,3


In [13]:
# getting dummies 
df_ratings_expanded = pd.get_dummies(data=df_ratings_expanded, columns=[ 'time_of_day', 'weekday'])

## D) Net Promoter Score

The Net Promoter Score is an index ranging from -100 to 100 that measures the willingness of customers to recommend a company's products or services to others. It is used as a proxy for gauging the customer's overall satisfaction with a company's product or service and the customer's loyalty to the brand. (Fred Heichheld - 2003)

Each movie, using NPS scale, will be graded with the value:
* -1 --> when given rating is equal or greater than 3.0 
*  0 --> for neutral evaluated movies with given rating of 3.5 and 4.0 
* +1 --> movies rating above or iqual to 4.5

The NPS for each movie will be calculated using the sum of the given ratings

In [14]:
# adding net promote score to each relation user-movie using ratings
conditionlist = [
    (df_ratings_expanded['rating'] >=4.5) ,
    (df_ratings_expanded['rating'] > 3) & (df_ratings_expanded['rating'] <4.5),
    (df_ratings_expanded['rating'] <= 3)]
choicelist = [1, 0, -1]
df_ratings_expanded['NPS'] = np.select(conditionlist, choicelist, default= 0)

# checking results
df_ratings_expanded

,userId,movieId,rating,hour,year_rated,time_of_day_1,time_of_day_2,time_of_day_3,time_of_day_4,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,NPS
0,1,296,5.0,15,2006,0,1,0,0,0,0,0,1,0,0,0,1
1,1,306,3.5,12,2006,0,1,0,0,0,0,0,1,0,0,0,0
2,1,307,5.0,12,2006,0,1,0,0,0,0,0,1,0,0,0,1
3,1,665,5.0,15,2006,0,1,0,0,0,0,0,1,0,0,0,1
4,1,899,3.5,12,2006,0,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25000090,162541,50872,4.5,21,2009,0,0,1,0,0,0,1,0,0,0,0,1
25000091,162541,55768,2.5,20,2009,0,0,1,0,0,0,1,0,0,0,0,-1
25000092,162541,56176,2.0,20,2009,0,0,1,0,0,0,1,0,0,0,0,-1
25000093,162541,58559,4.0,21,2009,0,0,1,0,0,0,1,0,0,0,0,0


## E) Adding IMDB date information

In [15]:
# importing data
df_imdb_year = pd.read_csv('df_imdb_year.csv')

# merging datasets
df_ratings_expanded = pd.merge(df_ratings_expanded, df_imdb_year, on='movieId', how='inner')

# converting year columns into interger
df_ratings_expanded['year_rated'] = df_ratings_expanded['year_rated'].astype(int)

# creating a column to calculate the age of the movie when rated
df_ratings_expanded['age_movie_when_rated'] = df_ratings_expanded['year_rated'] - df_ratings_expanded['year_imdb']

# checking results
df_ratings_expanded

,userId,movieId,rating,hour,year_rated,time_of_day_1,time_of_day_2,time_of_day_3,time_of_day_4,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,NPS,year_imdb,age_movie_in_2019,age_movie_when_rated
0,1,296,5.0,15,2006,0,1,0,0,0,0,0,1,0,0,0,1,1994,25,12
1,3,296,5.0,14,2015,0,1,0,0,0,0,0,0,1,0,0,1,1994,25,21
2,4,296,4.0,21,2019,0,0,1,0,0,0,0,0,0,0,1,0,1994,25,25
3,5,296,4.0,13,1996,0,1,0,0,0,1,0,0,0,0,0,0,1994,25,2
4,7,296,4.0,11,1996,1,0,0,0,0,0,0,0,0,0,1,0,1994,25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24993489,162358,200192,2.0,18,2019,0,0,1,0,1,0,0,0,0,0,0,-1,1986,33,33
24993490,162358,200194,2.0,18,2019,0,0,1,0,1,0,0,0,0,0,0,-1,2003,16,16
24993491,162386,139970,3.5,17,2019,0,1,0,0,1,0,0,0,0,0,0,0,1995,24,24
24993492,162386,200726,4.0,15,2019,0,1,0,0,1,0,0,0,0,0,0,0,1995,24,24


In [16]:
# dropping no longer necessary columns
df_ratings_expanded.drop(['year_imdb','hour','year_rated'], inplace=True, axis=1)

# storing intermediate results on csv file
df_ratings_expanded.reset_index(drop=True)
df_ratings_expanded.to_csv('df_ratings_by_time.csv', index=False)

## F) Resuming ratings by movieId

In [17]:
# shortcut | reading from stored intermediate results
# df_ratings_expanded = pd.read_csv('df_ratings_by_time.csv')

In [18]:
# columns summarized by count
df_movie_ratings_count = df_ratings_expanded[['movieId', 'userId']] 
df_movie_ratings_count = df_movie_ratings_count.groupby(['movieId']).count().reset_index()

# columns summarized by average
df_movies_ratings_mean = df_ratings_expanded[['movieId', 'rating', 'age_movie_in_2019', 'age_movie_when_rated']]
df_movies_ratings_mean = df_movies_ratings_mean.groupby(['movieId']).mean().reset_index()

# columns summarized by sum
df_movies_ratings_sum = df_ratings_expanded[['movieId', 'NPS','time_of_day_1','time_of_day_2','time_of_day_3','time_of_day_4','weekday_0','weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6']]
df_movies_ratings_sum = df_movies_ratings_sum.groupby(['movieId']).sum().reset_index()

# merging results
df_movies_voting_profile = df_movie_ratings_count.merge(df_movies_ratings_mean, on='movieId', how='left')
df_movies_voting_profile = df_movies_voting_profile.merge(df_movies_ratings_sum, on='movieId', how='left')
df_movies_voting_profile = df_movies.merge(df_movies_voting_profile, on='movieId', how='outer')

# checking results
df_movies_voting_profile.head(2)

,movieId,title,userId,rating,age_movie_in_2019,age_movie_when_rated,NPS,time_of_day_1,time_of_day_2,time_of_day_3,time_of_day_4,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,1,Toy Story,57309.0,3.893708,24.0,11.041425,4713.0,10574.0,15071.0,18313.0,13351.0,8308.0,8887.0,8671.0,8151.0,7595.0,7965.0,7732.0
1,2,Jumanji,24228.0,3.251527,24.0,10.039830,-10415.0,4749.0,6574.0,7689.0,5216.0,3522.0,3557.0,3654.0,3597.0,3226.0,3379.0,3293.0


In [19]:
# storing intermediate results on csv file
df_movies_voting_profile = df_movies_voting_profile.reset_index(drop=True)
df_movies_voting_profile.to_csv('df_movies_movielens.csv', index=False)

## G)  Creating movies dataset for Kmeans Clustering

In [20]:
# shortcut | reading from stored intermediate results
#df_imdb_expanded = pd.read_csv('df_imdb_expanded.csv')
#df_movies_voting_profile = pd.read_csv('df_movies_movielens.csv')

### Joining movies data from IMDB and MovieLens 

In [21]:
# merging imdb and movielens datasets
df_movies_4kmeans = df_movies_voting_profile.merge(df_imdb_expanded, on='movieId', how='inner')

# droping 3375 not found on IMDB database
df_movies_4kmeans = df_movies_4kmeans.dropna()

### Data wrangling for clustering

In [22]:
# Renaming columns
df_movies_4kmeans.rename(columns = {'userId':'voting_count', 'rating':'avg_rate'}, inplace = True)
df_movies_4kmeans['imdbVotes'] = df_movies_4kmeans['imdbVotes'].str.replace(',','').astype(int)

# checking results
df_movies_4kmeans.head(2)

,movieId,title,voting_count,avg_rate,age_movie_in_2019,age_movie_when_rated,NPS,time_of_day_1,time_of_day_2,time_of_day_3,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,1,Toy Story,57309.0,3.893708,24.0,11.041425,4713.0,10574.0,15071.0,18313.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,24228.0,3.251527,24.0,10.039830,-10415.0,4749.0,6574.0,7689.0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# storing results in csv file
df_movies_4kmeans = df_movies_4kmeans.reset_index(drop=True)
df_movies_4kmeans.to_csv('df_movies_4kmeans.csv', index=False)

# checking results
df_movies_4kmeans.head(2)

,movieId,title,voting_count,avg_rate,age_movie_in_2019,age_movie_when_rated,NPS,time_of_day_1,time_of_day_2,time_of_day_3,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,1,Toy Story,57309.0,3.893708,24.0,11.041425,4713.0,10574.0,15071.0,18313.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,24228.0,3.251527,24.0,10.039830,-10415.0,4749.0,6574.0,7689.0,...,0,0,0,0,0,0,0,0,0,0


## H) Resuming ratings by userId

In [24]:
# importing data
df_user_ratings = pd.read_csv('df_ratings_by_time.csv')

# columns summarized by count
df_user_ratings_count = df_user_ratings[['movieId', 'userId']] 
df_user_ratings_count = df_user_ratings_count.groupby(['userId']).count().reset_index()

# columns summarized by average
df_user_ratings_mean = df_user_ratings[['userId', 'rating', 'age_movie_in_2019', 'age_movie_when_rated']]
df_user_ratings_mean = df_user_ratings_mean.groupby(['userId']).mean().reset_index()

# columns summarized by sum
df_user_ratings_sum = df_user_ratings[['userId', 'NPS','time_of_day_1',	'time_of_day_2','time_of_day_3','time_of_day_4','weekday_0','weekday_1','weekday_2',	'weekday_3','weekday_4','weekday_5','weekday_6']]
df_user_ratings_sum = df_user_ratings_sum.groupby(['userId']).sum().reset_index()

# merging results
df_user_ratings = df_user_ratings_count.merge(df_user_ratings_mean, on='userId', how='left')
df_user_ratings = df_user_ratings.merge(df_user_ratings_sum, on='userId', how='left')

# storing user profile results
df_user_ratings.to_csv('df_users_profile.csv')

#checking results
df_user_ratings

,userId,movieId,rating,age_movie_in_2019,age_movie_when_rated,NPS,time_of_day_1,time_of_day_2,time_of_day_3,time_of_day_4,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,1,70,3.814286,33.657143,20.657143,7,0,70,0,0,0,0,0,70,0,0,0
1,2,184,3.630435,25.891304,12.891304,25,0,0,184,0,0,0,0,0,0,184,0
2,3,656,3.697409,14.434451,11.641768,-83,0,501,0,155,155,0,0,90,402,9,0
3,4,242,3.378099,14.090909,14.090909,-32,0,0,240,2,0,3,6,0,0,0,233
4,5,101,3.752475,27.524752,5.049505,-17,6,37,57,1,10,90,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162536,162537,101,4.039604,27.118812,11.118812,12,0,0,0,101,101,0,0,0,0,0,0
162537,162538,154,3.415584,19.844156,15.844156,-12,0,154,0,0,0,0,0,154,0,0,0
162538,162539,47,4.510638,36.319149,18.319149,25,0,0,47,0,0,0,0,0,0,0,47
162539,162540,88,3.829545,16.431818,6.431818,17,88,0,0,0,0,0,0,76,0,12,0
